In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from clickhouse_driver import Client
from IPython.display import display

client = Client(host='',database="february2024",user="",password="",
client_name="python-driver from avisoiu")


def write_day_output(date):
    display("Querying " + date + "\n")
    output = client.query_dataframe("""
        WITH
        each_packet AS (
            SELECT
                SrcIP,
                Timestamp,
                DstIP,
                DstPort,
                SrcPort,
                Payload,
                any(Timestamp) OVER (PARTITION BY SrcIP ORDER BY Timestamp ASC ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING) as PrevTimestamp,
                any(DstIP) OVER (PARTITION BY SrcIP ORDER BY Timestamp ASC ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING) as PrevDstIP,
                if(toDate(PrevTimestamp) = '1970-01-01', NULL, age('ms', PrevTimestamp, Timestamp)) as TimeDiff,
                if(PrevDstIP = 0, NULL, DstIP - PrevDstIP) as DistanceToPrevDest,
                multiIf(
                    -- zmap - IPId static to 54321 - survey
                    IPId = 54321,
                    1,
                    -- massscan - IPId = dest addr xor dest port xor tcp seq - survey
                    IPId = bitXor(DstIP, bitXor(DstPort, Seq)),
                    2,
                    -- hajime - attack botnet
                    Window = 14600,
                    3,
                    -- Mozi botnet
                    Window = 29040 OR Window = 14520,
                    4,
                    -- atk03
                    bitXor(DstIP, Seq) = 3232235778 AND Window = 1300,
                    5,
                    -- atk05
                    Seq = 333994513,
                    6,
                    -- atk06
                    Seq = 30000 AND Window = 65535,
                    7,
                    -- surv01
                    Window = 0,
                    8,
                    -- surv02
                    Seq = 100 AND IPId = 123 AND Window = 1024,
                    9,
                    -- Mirai
                    bitXor(DstIP, Seq) = 0,
                    10,
                    -- other
                    11
                ) as tool_fingerprint
            FROM tcppackets
            WHERE SYN = true
                    AND ACK = False
                    AND toDate(Timestamp) = '"""+date+"""'
        )
    SELECT SrcIP as src_ip,
           -- 4294967040 - 255.255.255.0 subnet mask
           toUInt32(bitAnd(src_ip, 4294967040)) as subnet,
           i.ASN as asn,
           i.Country as country,
           i.City as city,
           age('s', min(Timestamp), max(Timestamp)) as scan_length_seconds,
           toHour(min(Timestamp)) as start_hour,
           toHour(max(Timestamp)) as end_hour,
           median(TimeDiff) as median_time_diff,
           COUNT(DISTINCT SrcPort) as distinct_src_ports,
           COUNT(DISTINCT DstPort) as distinct_dest_ports,
           arrayElement(topK(1)(toUInt32(DstPort)), 1) as top_port,
           quantileExact(0.25)(DistanceToPrevDest) as q1_prev_ip,
           median(DistanceToPrevDest) AS median_prev_ip,
           quantileExact(0.75)(DistanceToPrevDest) as q3_prev_ip,
           COUNT(DISTINCT DstIP) as distinct_ips,
           COUNT(*) as total_hits,
           total_hits/distinct_ips as probes_per_ip,
           COUNT(DISTINCT tool_fingerprint) as distinct_fingerprints,
           arrayElement(topK(1)(tool_fingerprint), 1) as top_fingerprint,
           COUNT(DISTINCT Payload) as distinct_payloads,
           avg(length(Payload)) as avg_payload_length,
           arrayElement(topK(1)(Payload), 1) as favorite_payload,
           abs(q3_prev_ip) - abs(q1_prev_ip) as q1q3_delta,
           multiIf(
            -- pure sequential
            median_prev_ip = 1 AND q1_prev_ip = 1 AND q3_prev_ip = 1,
            1,
            -- patterned sequential
            median_prev_ip = 1 AND q1q3_delta < 0.1 * abs(q1_prev_ip),
            2,
            -- jumpy sequential
            median_prev_ip = 1 AND q1q3_delta > 0.1 * abs(q1_prev_ip),
            3,
            -- slow
            median_prev_ip = 0 AND q1_prev_ip = 0 AND q3_prev_ip = 0,
            4,
            -- random uniform
            q1q3_delta < 0.1 * abs(q1_prev_ip),
            5,
            -- other
            6
           ) as generation_algorithm
    FROM each_packet p
    JOIN ipinfo i ON i.SrcIP = p.SrcIP AND toDate(i.Timestamp) = '"""+date+"""'
    GROUP BY SrcIP, i.ASN, i.Country, i.City
    HAVING total_hits > 1
    """)
    output.to_csv("day_data/"+date+".csv") 
    display(date + " written to csv\n")


In [2]:
empty_df = pd.DataFrame()
empty_df.to_csv("day_data/test.csv")

In [3]:
from datetime import datetime, timedelta 

def calculate_for_dates(start_date, num_days):
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    for _ in range(num_days):
        write_day_output(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1) 

display("Aggregating data by day...\n")
calculate_for_dates("2024-02-01", 20) 
display("Done!\n")



'Aggregating data by day...\n'

'Querying 2024-02-01\n'

'2024-02-01 written to csv\n'

'Querying 2024-02-02\n'

'2024-02-02 written to csv\n'

...

'Querying 2024-02-19\n'

'2024-02-19 written to csv\n'

'Querying 2024-02-20\n'

'2024-02-20 written to csv\n'

'Done!\n'